In [1]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# 패키지 import

In [2]:
# 필요한 패키지 임포트
!pip install sentence-transformers
!pip install faiss-cpu

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
import faiss
from sentence_transformers import SentenceTransformer, util
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# 1-1. Import Data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/ChecKHUMate/merge_domitory_data.csv')

# 1-2. Data preprocessing

In [4]:
def prepare_data(data):
    """
    주어진 데이터 프레임에서 필요한 정보를 추출하고 포맷팅하는 함수.

    Parameters:
        data (pd.DataFrame): 원본 데이터 프레임.

    Returns:
        user_data (pd.DataFrame): 사용자 데이터 프레임.
        wish_data (pd.DataFrame): 사용자의 선호에 관한 데이터 프레임.
    """
    # 사용자 데이터 선택
    user_data_columns = [
        'user_id', 'domitory', 'age', 'student_id', 'gender', 'major',
        'bedtime', 'clean_duration', 'smoke', 'alcohol', 'mbti', 'one_sentence'
    ]
    user_data = data[user_data_columns]
    user_data = user_data.set_index('user_id')

    # 사용자 선호 데이터 선택
    wish_data_columns = [
        'user_id', 'wish_domitory', 'wish_age', 'wish_student_id', 'wish_gender',
        'wish_major', 'wish_bedtime', 'wish_clean_duration', 'wish_smoke',
        'wish_alcohol', 'wish_mbti'
    ]
    wish_data = data[wish_data_columns]
    wish_data = wish_data.set_index('user_id')

    return user_data, wish_data

In [5]:
user_data, wish_data = prepare_data(data)

In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# 코랩용.
def encode_and_concat_to_array(dataframe, column_name):
    """
    원핫 인코딩을 수행하고 원본 데이터프레임에 결과를 추가한 후 전체를 numpy 배열로 반환합니다.

    Args:
    dataframe (pd.DataFrame): 원본 데이터를 포함한 데이터프레임.
    column_name (str): 원핫 인코딩을 적용할 열의 이름.

    Returns:
    numpy.ndarray: 원핫 인코딩된 결과와 원본 데이터가 결합된 numpy 배열.
    """
    # 원핫 인코더 초기화 및 피팅
    encoder = OneHotEncoder(sparse_output=False)  # 바로 numpy array 반환 설정
    encoded_data = encoder.fit_transform(dataframe[[column_name]])

    # 원핫 인코딩 결과를 DataFrame으로 변환
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))

    # 데이터프레임의 인덱스 리셋
    dataframe = dataframe.reset_index(drop=True)
    encoded_df = encoded_df.reset_index(drop=True)

    # 원본 데이터와 원핫 인코딩된 데이터를 결합
    new_dataframe = pd.concat([dataframe, encoded_df], axis=1)

    # 인코딩된 열 제거
    new_dataframe = new_dataframe.drop(column_name, axis=1)

    return new_dataframe

In [7]:
# DB 용
def encode_and_concat_to_array(dataframe, column_name):
    """
    원핫 인코딩을 수행하고 원본 데이터프레임에 결과를 추가한 후 전체를 numpy 배열로 반환합니다.

    Args:
    dataframe (pd.DataFrame): 원본 데이터를 포함한 데이터프레임.
    column_name (str): 원핫 인코딩을 적용할 열의 이름.

    Returns:
    numpy.ndarray: 원핫 인코딩된 결과와 원본 데이터가 결합된 numpy 배열.
    """
    # 원핫 인코더 초기화 및 피팅
    encoder = OneHotEncoder(sparse_output=False)  # 바로 numpy array 반환 설정
    encoded_data = encoder.fit_transform(dataframe[[column_name]])

    # 원핫 인코딩 결과를 DataFrame으로 변환
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([column_name]))

    # 원본 데이터와 원핫 인코딩된 데이터를 결합
    new_dataframe = pd.concat([dataframe, encoded_df], axis=1)

    new_dataframe = new_dataframe.drop(column_name, axis = 1)

    # 전체 데이터프레임을 numpy 배열로 변환
    final_array = new_dataframe.to_numpy()

    return final_array

In [9]:
user_df = encode_and_concat_to_array(user_data, 'mbti')
wish_df = encode_and_concat_to_array(wish_data, 'wish_mbti')

In [10]:
# Faiss 유사도 계산을 위한 데이터 전처리

# 1) user_data에서 one_sentence 데이터 drop
# 0행: 'user_id', 1행:'domitory', 2행: 'age', 3행: 'student_id', 4행: 'gender', 5행: 'major', 6행: 'bedtime', 7행: 'clean_duration', 8행:'smoke',
# 9행: 'alcohol', 10행: 'one_sentence'

# one_sentence drop
user_df_a = user_df.drop(columns = 'one_sentence')

# 2. Data Filtering & Modeling

## 2-1 Faiss 유사도

In [11]:
import numpy as np
import faiss

# 각 사용자의 wish_data를 기반으로 user_data에서 자기 자신을 제외한 상위 k개의 가장 유사한 사용자들을 찾아낸 결과를 배열로 반환합니다.
class Recommender_without_sentence:
    def __init__(self, user_data, wish_data):
        self.user_data = user_data
        self.wish_data = wish_data
        self.index = None
        self.d = user_data.shape[1]

    def build_index(self):
        """FAISS 인덱스를 생성하고 유저 데이터를 추가합니다."""
        self.index = faiss.IndexFlatL2(self.d)
        self.index.add(self.user_data.astype('float32'))

    def find_roommates(self, k=10):
        """각 유저의 wish_data에 대해 가장 유사한 유저를 찾되, 자기 자신은 제외하고, -1 인덱스가 발생한 경우도 기록합니다."""
        if self.index is None:
            self.build_index()

        all_results = []  # 결과를 저장할 배열
        invalid_indices_info = []  # -1 인덱스가 발생한 정보를 저장할 배열

        for i in range(self.wish_data.shape[0]):
            # 자기 자신을 포함하여 k+1개의 결과를 검색
            distances, indices = self.index.search(self.wish_data[i:i+1].astype('float32'), k+1)

            # 자기 자신의 인덱스와 -1을 제외
            valid_indices = indices[0][(indices[0] != i) & (indices[0] != -1)]

            # -1이 발생한 경우 기록
            if np.any(indices[0] == -1):
                invalid_indices_info.append((i, list(indices[0])))

            # k개의 결과만 반환
            all_results.append(valid_indices[:k])

        # 결과와 -1 정보 모두 반환
        return np.array(all_results)

In [12]:
recommender = Recommender_without_sentence(user_df_a, wish_df)
roommate_recommendations = recommender.find_roommates()

In [13]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 구버전
class IntroductionRecommender:
    def __init__(self, user_data):
        self.user_data = user_data
        self.embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
        self.index = None
        self.embeddings = None
        self.user_data['one_sentence'] = user_data['one_sentence'].fillna('No introduction provided').astype(str)

    def create_faiss_index(self):
        """임베딩을 생성하고 FAISS 인덱스에 추가합니다."""
        # 모든 자기소개 문장 임베딩 생성
        self.embeddings = self.embedder.encode(self.user_data['one_sentence'].tolist(), convert_to_tensor=False)

        # FAISS 인덱스 생성
        d = self.embeddings.shape[1]  # 임베딩 차원
        self.index = faiss.IndexFlatL2(d)
        self.index.add(self.embeddings.astype('float32'))

    def find_similar_introductions(self, result_indices, k=10):
        if self.index is None:
            raise ValueError("FAISS index is not built. Please run create_faiss_index method first.")

        all_group_similarities = []

        for indices in result_indices:
            min_distances = {}  # 인덱스별 최소 거리를 저장할 사전

            for element in indices:
                query_embedding = self.embeddings[element]
                D, I = self.index.search(query_embedding.reshape(1, -1), k+1)

                for i, d in zip(I[0], D[0]):
                    if i != element and i in indices:
                        if i not in min_distances or min_distances[i] > d:
                            min_distances[i] = d  # 최소 거리 업데이트

            # 인덱스와 거리를 튜플로 변환하여 리스트로 만든 후, 거리에 따라 정렬
            group_similarities = sorted(min_distances.items(), key=lambda x: x[1])
            all_group_similarities.append(group_similarities)

        return all_group_similarities

In [155]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 신버전
class IntroductionRecommender:
    def __init__(self, user_data):
        self.user_data = user_data
        self.embedder = SentenceTransformer('jhgan/ko-sroberta-multitask')
        self.index = None
        self.id_index = np.array(self.user_data.index)
        self.embeddings = None
        self.user_data['one_sentence'] = user_data['one_sentence'].fillna('No introduction provided').astype(str)

    def create_faiss_index(self):

        """임베딩을 생성하고 FAISS 인덱스에 추가합니다."""
        # 모든 자기소개 문장 임베딩 생성
        self.embeddings = self.embedder.encode(self.user_data['one_sentence'].tolist(), convert_to_tensor=False)

        normalized_embeddings = self.embeddings.copy()

        # normalized_embeddings의 벡터를 L2 Norm(유클리드 거리 기준)으로 정규화합니다.
        # 이렇게 함으로써, 벡터의 길이를 1로 만들어 줍니다. 이것은 내적 기반의 유사도 검색에서 중요한 절차입니다.
        # 왜냐하면 정규화된 벡터들 사이의 내적은 두 벡터의 코사인 유사도와 동일하기 때문입니다.
        faiss.normalize_L2(normalized_embeddings)

        # FAISS 인덱스 생성
        d = self.embeddings.shape[1]  # 임베딩 차원
        index_flat = faiss.IndexFlatIP(d)

        self.index =index = faiss.IndexIDMap(index_flat)
        self.index.add_with_ids(normalized_embeddings, self.id_index)


    def find_similar_introductions(self, result_indices, k=10):
        if self.index is None:
            raise ValueError("FAISS index is not built. Please run create_faiss_index method first.")

        all_group_similarities = []

        for indices in result_indices:
            min_distances = {}  # 인덱스별 최소 거리를 저장할 사전

            for element in indices:
                # 인덱스에 해당하는 텍스트만 인코딩합니다.
                text_to_encode = [self.user_data['one_sentence'].iloc[element]]
                vector = self.embedder.encode(text_to_encode)
                faiss.normalize_L2(vector)

                D, I = self.index.search(vector.reshape(1, -1), k+1)

                for i, d in zip(I[0], D[0]):
                    if i != element and i in indices:
                        if i not in min_distances or min_distances[i] > d:
                            min_distances[i] = d  # 최소 거리 업데이트


            # 인덱스와 거리를 튜플로 변환하여 리스트로 만든 후, 텍스트 유사도에 따라
            group_similarities = sorted(min_distances.items(), key=lambda x: x[1], reverse = True)
            all_group_similarities.append(group_similarities)

        return all_group_similarities

In [157]:
recommender = IntroductionRecommender(user_df)
recommender.create_faiss_index()
similarities = recommender.find_similar_introductions(roommate_recommendations)
print(similarities[102])

[(100, 0.6553358), (106, 0.6553357), (103, 0.48954824), (97, 0.48547697), (88, 0.48547685), (107, 0.4410506), (105, 0.41970214), (94, 0.3905151)]


In [156]:
roommate_recommendations[102]

array([ 88, 103, 113,  94, 100, 105, 106,  96,  97, 107])

In [153]:
user_data.iloc[100]

domitory                                      2
age                                           4
student_id                                    4
gender                                        0
major                                         1
bedtime                                       3
clean_duration                                1
smoke                                         0
alcohol                                       1
mbti                                       ISFP
one_sentence      방에 타인이 들어오는 걸 별로 안 좋아하는 편입니다.
Name: 101, dtype: object

In [158]:
wish_data.iloc[100]

wish_domitory             2
wish_age                  4
wish_student_id           4
wish_gender               0
wish_major                1
wish_bedtime              3
wish_clean_duration       1
wish_smoke                0
wish_alcohol              1
wish_mbti              ENTP
Name: 101, dtype: object

In [152]:
user_data.iloc[102]

domitory                       2
age                            4
student_id                     4
gender                         0
major                          2
bedtime                        3
clean_duration                 0
smoke                          0
alcohol                        0
mbti                        ISTP
one_sentence      독립성이 강한 성격입니다.
Name: 103, dtype: object